# Word Embeddings: Encoding Lexical Semantics





Two examples in this notebook are considered:

1.   N-Gram Language Modeling
2.   Continuous Bag-of-Words

Based on a notebook of Robert Guthrie.





Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the [distributional
hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics).

## Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

$$
q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]
$$

$$
q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]
$$

Then we can get a measure of similarity between these words by doing:
$$
\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}
$$

Although it is more common to normalize by the lengths:

$$
\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)
$$
where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.

## Word Embeddings in Pytorch

Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).

In [1]:
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(lookup_tensor)
print(hello_embed)

tensor([0])
tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)


In [4]:
[word_to_ix["hello"]]

[0]

## First Example: N-Gram Language Modeling


Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute
$$
P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )
$$
where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.





In [5]:

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 5, just so you can see what they look like
print(trigrams[:5])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(word_to_ix)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow,')]
{'art': 0, "excuse,'": 1, 'on': 2, 'made': 3, 'more': 4, 'thou': 5, 'small': 6, 'shall': 7, 'all-eating': 8, 'livery': 9, "feel'st": 10, 'his': 11, 'child': 12, 'all': 13, 'in': 14, 'fair': 15, 'How': 16, 'held:': 17, 'winters': 18, 'To': 19, 'were': 20, 'beauty': 21, 'Were': 22, "'This": 23, 'gazed': 24, 'new': 25, 'couldst': 26, 'warm': 27, 'trenches': 28, 'praise.': 29, 'thine!': 30, "totter'd": 31, 'old': 32, 'And': 33, 'Shall': 34, 'treasure': 35, 'when': 36, 'shame,': 37, 'dig': 38, 'it': 39, 'If': 40, 'the': 41, 'thy': 42, 'a': 43, 'to': 44, 'This': 45, 'being': 46, 'brow,': 47, 'of': 48, 'Then': 49, 'much': 50, 'see': 51, 'Thy': 52, 'my': 53, "beauty's": 54, "deserv'd": 55, 'count,': 56, 'Will': 57, 'Proving': 58, 'field,': 59, 'lies,': 60, 'days;': 61, 'proud': 62, 'eyes,': 63, 'When': 64, 'blood': 65, 'thriftless': 

### Question: what is the role of "nn.Embedding"?

It is a lookup table: it transform an integer (index of a word) into a vector (word vector).

In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[520.3021912574768, 517.7363927364349, 515.1875436306, 512.654422044754, 510.13636898994446, 507.63098788261414, 505.13819670677185, 502.6569356918335, 500.185263633728, 497.72437024116516]


In [7]:
print(context)
print(context_idxs)

["feel'st", 'it']
tensor([10, 39])


## Second Example: Continuous Bag-of-Words


The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.



### Question: what is the role of "word_to_ix" and "ix_to_word"?

It transforms a word (as a string) into an index (as an integer) or reciprocally.

In [8]:
import torch
import torch.nn as nn
import numpy as np

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

def get_index_of_max(input):
    index = 0
    for i in range(1, len(input)):
        if input[i] > input[index]:
            index = i 
    return index

def get_max_prob_result(input, ix_to_word):
    return ix_to_word[get_index_of_max(input)]


CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 100

word_to_ix = {}
ix_to_word = {}

raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
print(vocab_size)

for i, word in enumerate(vocab):
    word_to_ix[word] = i
    ix_to_word[i] = word



49


In [9]:
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
    
    
print(context)    

['the', 'computer', 'our', 'spells.']


Class CBOW

### Question: what is the difference between the CBOW architecture coded in the following CBOW class and the CBOW architecture described in the lecture?

There is an extra-linear layer.

In [10]:
class CBOW(torch.nn.Module):

    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, 128)

        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)

        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_embedding(self, word):
        word = torch.LongTensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)





Train the CBOW model

In [11]:
model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


for epoch in range(50):
    total_loss = 0
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  
        model.zero_grad()
        log_probs = model(context_vector)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        loss.backward()
        optimizer.step()

        total_loss += loss.data


Test the model

In [12]:
# ====================== TEST
context = ['People','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector).data.numpy()
print('Raw text: {}\n'.format(' '.join(raw_text)))
print('Context: {}\n'.format(context))
print('Prediction: {}'.format(get_max_prob_result(a[0], ix_to_word)))

Raw text: We are about to study the idea of a computational process. Computational processes are abstract beings that inhabit computers. As they evolve, processes manipulate other abstract things called data. The evolution of a process is directed by a pattern of rules called a program. People create programs to direct processes. In effect, we conjure the spirits of the computer with our spells.

Context: ['People', 'create', 'to', 'direct']

Prediction: programs


### Question: How do we interpret the output "a" of "model"?

This is the output of the network for the context. It corresponds to the activity values of each possible word.

In [13]:
print(a)
print(a.shape)

[[-6.9875774 -7.5215244 -6.164543  -5.68129   -6.2408166 -7.162094
  -6.733523  -6.0839834 -3.9267876 -6.8135343 -4.053952  -8.33693
  -6.413438  -6.78292   -5.773486  -5.612984  -6.163023  -4.0328693
  -6.318073  -4.978421  -5.4953775 -4.140299  -6.59852   -7.6728935
  -3.7340837 -0.2316637 -7.7967415 -6.2310157 -4.943166  -5.327487
  -7.9878592 -6.330804  -5.9505544 -6.47496   -7.041203  -4.144412
  -5.536264  -6.7150087 -7.370602  -4.717738  -4.6652865 -7.2030854
  -7.176071  -5.854429  -8.069258  -6.253221  -6.969037  -7.342703
  -6.3618426]]
(1, 49)


### Question: what is the word embedding of the word "People"?

In [14]:
print(model.get_word_embedding('People'))
model.get_word_embedding('People').shape

tensor([[-8.5619e-01, -3.1068e-01, -1.7637e+00, -1.4132e+00,  8.7400e-01,
         -7.7914e-01, -1.8653e+00,  2.2099e-01, -1.3672e+00, -7.8361e-03,
         -5.5736e-01, -1.2103e-01, -9.8837e-01,  1.0450e+00, -1.8874e+00,
         -1.0105e+00,  5.3073e-01, -9.3289e-01, -2.9664e-01, -1.0145e+00,
         -1.2755e+00,  1.5743e+00, -6.6098e-01, -7.1793e-01, -4.9651e-01,
         -3.8567e-01,  9.7484e-01,  2.9868e-01,  1.8145e+00, -2.2098e-01,
          9.7299e-01,  1.3633e+00,  1.1662e+00,  9.6197e-01,  9.3213e-01,
          6.2440e-02, -9.6806e-01,  5.4639e-01, -4.4423e-01,  1.3129e+00,
          1.8879e+00, -3.9610e-01,  1.3823e+00, -1.1172e+00,  1.4643e-01,
         -1.4804e+00, -4.9700e-01,  8.1832e-01, -1.4929e+00, -1.0792e+00,
         -8.4384e-01,  1.2267e+00, -4.6320e-01, -1.1835e+00, -2.0542e+00,
         -1.4472e+00, -3.4112e-01, -2.7993e+00, -3.2661e-01,  1.0935e+00,
         -3.9648e-01, -1.1879e+00, -2.7184e+00, -3.5597e-01, -7.2795e-02,
         -5.6070e-01,  7.9183e-01,  7.

torch.Size([1, 100])

### Question: in the lecture, the CBOW model can propose at least two kinds of embedding for a given word. Which one is used in this code?

Here, we use a deep neural network with ReLU function. It ia a deep word embedding.

### Question: count all the parameters of the model. Is the embedding matrix learned during the training step?

In [15]:
# Function to count the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [16]:
print(model.parameters)
print("\nTotal number of parameters {}\n".format(count_parameters(model)))

<bound method Module.parameters of CBOW(
  (embeddings): Embedding(49, 100)
  (linear1): Linear(in_features=100, out_features=128, bias=True)
  (activation_function1): ReLU()
  (linear2): Linear(in_features=128, out_features=49, bias=True)
  (activation_function2): LogSoftmax(dim=-1)
)>

Total number of parameters 24149



Details on the computation:

In [17]:
49*100+100*128+128*49+128+49

24149